In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, Callback, ModelCheckpoint
import numpy as np
import pandas as pd
from collections import Counter
import re, os, sys, csv, math, operator

/home/vgf3011/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vgf3011/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vgf3011/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vgf3011/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
architecture = '1024x4D-512x3D-256x3D-128x3D-64x2-32x1-1'
activation = 'relu'
dropouts = [0.8, 0.9, 0.7, 0.8]
SEED = 1234567
num_input = 86

In [3]:
def define_model(data, architecture, num_labels=1, activation='relu', dropouts=[]):

        assert '-' in architecture
        archs = architecture.strip().split('-')
        net = data
        pen_layer = net
        prev_layer = net
        prev_num_outputs = None
        prev_block_num_outputs = None
        prev_stub_output = net
        for i in range(len(archs)):
            arch = archs[i]
            if 'x' in arch:
                arch = arch.split('x')
                num_outputs = int(re.findall(r'\d+',arch[0])[0])
                layers = int(re.findall(r'\d+',arch[1])[0])
                j = 0
                aux_layers = re.findall(r'[A-Z]',arch[0])
                for l in range(layers):
                    if aux_layers and aux_layers[0] == 'B':
                        if len(aux_layers)>1 and aux_layers[1]=='A':
                            print('adding fully connected layers with %d outputs followed by batch_norm and act' % num_outputs)

                            net = Dense(num_outputs, 
                                        name='fc' + str(i) + '_' + str(j),
                                        activation=None)(net)
                            net = BatchNormalization(center=True, scale=True, name='fc_bn'+str(i)+'_'+str(j))(net)
                            if activation =='relu': net = Activation('relu')(net)
                        else:
                            print('adding fully connected layers with %d outputs followed by batch_norm' % num_outputs)
                            net = Dense(num_outputs,
                                        name='fc' + str(i) + '_' + str(j),
                                        activation=activation)(net)
                            net = BatchNormalization(center=True, scale=True,
                                             name='fc_bn' + str(i) + '_' + str(j))(net)

                    else:
                        print('adding fully connected layers with %d outputs' % num_outputs)

                        net = Dense(num_outputs,
                                    name='fc' + str(i) + '_' + str(j), 
                                    activation=activation)(net)

                    if 'R' in aux_layers:
                        if prev_num_outputs and prev_num_outputs==num_outputs:
                            print('adding residual, both sizes are same')

                            net = net+prev_layer
                        else:
                            print('adding residual with fc as the size are different')
                            net = net + Dense(num_outputs,
                                                name='fc' + str(i) + '_' +'dim_'+ str(j),
                                                activation=None)(prev_layer)
                    prev_num_outputs = num_outputs
                    j += 1
                    prev_layer = net
                aux_layers_sub = re.findall(r'[A-Z]', arch[1])
                if 'R' in aux_layers_sub:
                    if prev_block_num_outputs and prev_block_num_outputs == num_outputs:
                        print('adding residual to stub, both sizes are same')
                        net = net + prev_stub_output
                    else:
                        print('adding residual to stub with fc as the size are different')
                        net = net + Dense(num_outputs,
                                         name='fc' + str(i) + '_' + 'stub_dim_' + str(j),
                                         activation=None)(prev_stub_output)

                if 'D' in aux_layers_sub and (num_labels == 1) and len(dropouts) > i:
                    print('adding dropout', dropouts[i])
                    net = Dropout(1.-dropouts[i], seed=SEED)(net, training=False)
                prev_stub_output = net
                prev_block_num_outputs = num_outputs
                prev_layer = net

            else:
                if 'R' in arch:
                    act_fun = 'relu'
                    print('using ReLU at last layer')  
                else:
                    act_fun = None
                pen_layer = net
                print('adding final layer with ' + str(num_labels) + ' output')
                net = Dense(num_labels, name='fc' + str(i),
                            activation=act_fun)(net)

        return net

In [4]:
inputs = Input(shape=(num_input,), name='elemental_fractions')
outputs = define_model(inputs, architecture, dropouts=dropouts)
model = Model(inputs=inputs, outputs=outputs, name= 'ElemNet')
model.summary(print_fn=lambda x: print(x))

print("Loaded model from disk")

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding fully connected layers with 1024 outputs
adding dropout 0.8
adding fully connected layers with 512 outputs
adding fully connected layers with 512 outputs
adding fully connected layers with 512 outputs
adding dropout 0.9
adding fully connected layers with 256 outputs
adding fully connected layers with 256 outputs
adding fully connected layers with 256 outputs
adding dropout 0.7
adding fully connected layers with 128 outputs
adding fully connected layers with 128 outputs
adding fully connected layers with 128 outputs
adding dropout 0.8
adding fully connected layers with 64 outputs
adding fully connected layers with 64 outputs
adding fully connected layers with 32 outputs
adding final layer with 1 output
Model: "ElemNet"
_________________________________________________________________
Layer (type)                 Output Shape              

In [5]:
#Contains 86 elements (Without Noble elements as it does not forms compounds in normal condition)
elements = ['H','Li','Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl',
            'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe','Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge',
            'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd',
            'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
            'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er','Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 
            'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu' ]

elements_all = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 
                'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni',
                'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 
                'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe',
                'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho',
                'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
                'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np',
                'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg',
                'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn']

# Regex to Choose from Element Name, Number and Either of the Brackets
token = re.compile('[A-Z][a-z]?|\d+|[()]')

# Create a dictionary with the Name of the Element as Key and No. of elements as Value
def count_elements(formula):
    tokens = token.findall(str(formula))
    stack = [[]]
    for t in tokens:
        if t.isalpha():
            last = [t]
            stack[-1].append(t)
        elif t.isdigit():
             stack[-1].extend(last*(int(t)-1))
        elif t == '(':
            stack.append([])
        elif t == ')':
            last = stack.pop()
            stack[-1].extend(last)   
    return dict(Counter(stack[-1]))

#Normalize the Value of the Dictionary
def normalize_elements(dictionary):
    dic_val = sum(dictionary.values()) 
    if dic_val == 0:
        factor = 0
    else:    
        factor=1.0/ dic_val  
        
    for k in dictionary:
        dictionary[k] = dictionary[k]*factor
    return dictionary

def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

print(Diff(elements_all, elements)) 

def elemental_fraction(dataframe):
    print('The loaded dataset has %d entries'%len(dataframe['pretty_comp']))

    compounds = dataframe['pretty_comp']

    print('The reduced dataset has %d entries'%len(compounds))
    
    compounds = [count_elements(x) for x in compounds]
    compounds = [normalize_elements(x) for x in compounds]

    in_elements = np.zeros(shape=(len(compounds), len(elements)))
    comp_no = 0

    for compound in compounds:
        keys = compound.keys()
        for key in keys:
            in_elements[comp_no][elements.index(key)] = compound[key]
        comp_no+=1  
    
    data = in_elements
    
    return data




['Cn', 'Cf', 'Fm', 'Fr', 'Po', 'Md', 'Ar', 'Rf', 'Ds', 'Hs', 'Rn', 'Db', 'Lr', 'No', 'Am', 'At', 'Ra', 'Bk', 'Sg', 'Es', 'He', 'Bh', 'Mt', 'Cm', 'Rg', 'Ne']


In [27]:
pred = pd.read_csv(r'../../data/sample_data.csv') 
print(pred.shape)    

pred_prop = pred[elements]

#train_phy['compound possible'] = train_phy['compound possible'].astype(float)

new_x_pred = pred_prop.values

new_x_pred = np.asarray(new_x_pred, dtype=np.float)

y_pred = pred.pop('delta_e').to_frame()

new_y_pred = np.array(y_pred)

new_y_pred.shape = (len(new_y_pred),)

(557, 273)


In [28]:
extractor = tf.keras.models.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers])
features = extractor(new_x_pred)
len(features)
#composition = pred.pop('composition').to_frame()
composition = pred.pop('pretty_comp').to_frame()

In [29]:
for i in range(len(features)):
    globals()["f" + str(i)] = features[i].numpy()
    globals()["df" + str(i)] = pd.DataFrame(data=globals()["f" + str(i)])
    globals()["ndf" + str(i)] = pd.concat([composition,globals()["df" + str(i)],y_pred], axis=1)
    path = '../data/sample_f{}.csv'.format(i)
    globals()["ndf" + str(i)].to_csv(path, index=False)

In [50]:
#inlayer = [1024, 1024, 1024, 1024, 512, 512, 512, 256, 256, 256, 128, 128, 128, 64, 64, 32]
#for i in range(len(features)):
#    globals()["f" + str(i)] = features[i].numpy()
#    globals()["df" + str(i)] = pd.DataFrame(data=globals()["f" + str(i)])
#    globals()["ndf" + str(i)] = pd.concat([composition,globals()["df" + str(i)],y_pred], axis=1)
#    path = "../data/sample_test_set.csv".format(i+1, inlayer[i])
#    globals()["ndf" + str(i)].to_csv(path, index=False)